In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))  
from utils.load_config import load_cfg
from utils.minio_client import MinioClient
import pandas as pd
from io import StringIO

In [2]:
config = load_cfg('../config/config.yaml')
minio_config = config['minio']
processed_folder_path = config['dataset']['processed_path']

In [3]:
minio_client = MinioClient(minio_config)

# get sell_prices on minio
sell_prices = minio_client.get_file(
    bucket_name=minio_config['bucket_name'],
    object_name='sell_prices'
)

sell_prices_df = pd.read_csv(StringIO(sell_prices))
print(sell_prices_df.head())

Bucket "m5-forecasting-bucket" already exists.
  store_id        item_id  wm_yr_wk  sell_price
0     CA_1  HOBBIES_1_001     11325        9.58
1     CA_1  HOBBIES_1_001     11326        9.58
2     CA_1  HOBBIES_1_001     11327        8.26
3     CA_1  HOBBIES_1_001     11328        8.26
4     CA_1  HOBBIES_1_001     11329        8.26


In [4]:
len(sell_prices_df)

6841121

# For each store id, get only 10000 first rows

In [5]:
filtered_sell_price_df = sell_prices_df.groupby('store_id').head(10000).reset_index(drop=True)
filtered_sell_price_df.groupby('store_id').size()

store_id
CA_1    10000
CA_2    10000
CA_3    10000
CA_4    10000
TX_1    10000
TX_2    10000
TX_3    10000
WI_1    10000
WI_2    10000
WI_3    10000
dtype: int64

In [6]:
filtered_sell_price_df['state'] = filtered_sell_price_df['store_id'].map(lambda x: x.split('_')[0])
filtered_sell_price_df['store_in_state'] = filtered_sell_price_df['store_id'].map(lambda x: x.split('_')[1])
filtered_sell_price_df['category'] = filtered_sell_price_df['item_id'].map(lambda x: x.split('_')[0])
filtered_sell_price_df['subcategory'] = filtered_sell_price_df['item_id'].map(lambda x: x.split('_')[1])
filtered_sell_price_df['item_number'] = filtered_sell_price_df['item_id'].map(lambda x: x.split('_')[2])

filtered_sell_price_df.head()

,store_id,item_id,wm_yr_wk,sell_price,state,store_in_state,category,subcategory,item_number
0,CA_1,HOBBIES_1_001,11325,9.58,CA,1,HOBBIES,1,001
1,CA_1,HOBBIES_1_001,11326,9.58,CA,1,HOBBIES,1,001
2,CA_1,HOBBIES_1_001,11327,8.26,CA,1,HOBBIES,1,001
3,CA_1,HOBBIES_1_001,11328,8.26,CA,1,HOBBIES,1,001
4,CA_1,HOBBIES_1_001,11329,8.26,CA,1,HOBBIES,1,001


In [7]:
filtered_sell_price_df.isna().sum()

store_id          0
item_id           0
wm_yr_wk          0
sell_price        0
state             0
store_in_state    0
category          0
subcategory       0
item_number       0
dtype: int64

In [8]:
filtered_sell_price_df.to_parquet(f'../{processed_folder_path}/sell_prices.parquet', index=False)